In [3]:
from pathlib import Path

from bbtautau.HLTs import HLTs as bbtautau_HLTs

In [7]:
years = ["2022", "2023"]

HLTs = {
    "PNetBB": [
        "HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35",
        "HLT_AK8PFJet230_SoftDropMass40_PNetBB0p06",
    ],
    "PNetTauTau": [
        "HLT_AK8PFJet230_SoftDropMass40_PFAK8ParticleNetTauTau0p30",
        "HLT_AK8PFJet230_SoftDropMass40_PNetTauTau0p03",
    ],
    "PFJet": ["HLT_AK8PFJet420_MassSD30"],
    "QuadJet (JetMET)": [
        "HLT_QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65",
        "HLT_QuadPFJet103_88_75_15_PFBTagDeepJet_1p3_VBF2",
        "HLT_QuadPFJet103_88_75_15_DoublePFBTagDeepJet_1p3_7p7_VBF1",
    ],
    "QuadJet (ParkingHH)": [
        "HLT_PFHT280_QuadPFJet30_PNet2BTagMean0p55",
        "HLT_PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70",
    ],
    "SingleTau": ["HLT_LooseDeepTauPFTauHPS180_L2NN_eta2p1"],
    "Ditau": ["HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"],
    "Ditau+Jet": [
        "HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60",
    ],
    "Muon": ["HLT_IsoMu24", "HLT_Mu50"],
    "Mu-tau": ["HLT_IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1"],
    "EGamma": [
        "HLT_Ele30_WPTight_Gsf",
        "HLT_Ele115_CaloIdVT_GsfTrkIdT",
        "HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165",
        "HLT_Photon200",
    ],
    "e-tau": ["HLT_Ele24_eta2p1_WPTight_Gsf_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"],
    "MET": ["HLT_PFMET120_PFMHT120_IDTight"],
}

HLT_dict = {year: {} for year in years}
HLT_list = bbtautau_HLTs.hlt_list(as_str=False)

for year in years:
    for key, hlts in HLTs.items():
        HLT_dict[year][key] = []
        for hlt in hlts:
            for hlt_obj in HLT_list[year]:
                if hlt_obj.get_name() == hlt:
                    HLT_dict[year][key].append(hlt_obj)
                    break

used = [
    "PNetBB",
    "PFJet",
    "SingleTau",
    "Ditau",
    "DitauJet",
    "Muon",
    "Mu-tau",
    "EGamma",
    "e-tau",
    "MET",
]

In [9]:
for year in years:
    lines = []
    for key, hlts in HLT_dict[year].items():
        # first, multirow entry for key for number of HLTs
        for i, hlt in enumerate(hlts):
            line = []
            if i == 0:
                line.append(rf"\multirow{{{len(hlts)}}}{{*}}{{{key}}}")
                line.append(rf"\multirow{{{len(hlts)}}}{{*}}{{{hlt.dataset}}}")
            else:
                line.append("")
                line.append("")
            
            line.append(hlt.get_name(hlt_prefix=False).replace("_", r"\_"))

            for channel in ["hh", "hm", "he"]:
                if i == 0:
                    val = r"$\checkmark$" if (channel in hlt.channel and key in used) else r"$\times$"
                    line.append(rf"\multirow{{{len(hlts)}}}{{*}}{{{val}}}")
                else:
                    line.append("")
            
            if i == len(hlts) - 1:
                line[-1] += r" \\\hline"
            else:
                line[-1] += r" \\"

            lines.append(" & ".join(line))

    # remove trailing "\\\hline"
    lines[-1] = lines[-1][:-8]

    with Path(f"triggers/all_triggers_{year}.tex").open("w") as f:
        f.write("\n".join(lines))